In [2]:
# 이항 분류
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

In [8]:
from tabnanny import verbose
from streamlit import metric


x_data = np.array([[1,2], [2, 2], [3, 4], [4, 3], [3, 2], [2, 1]], dtype=np.float32)
y_data = np.array([[0], [0], [0], [1], [1], [1]], dtype=np.float32)

# 모델 정의 방법1: Sequential api
# model = Sequential([
#     Input(np.shape=(2,)),
#     Dense(units=1, activation='sigmoid')
# ])
model = Sequential()
model.add(Input(shape=(2,)))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

model.summary()

model.fit(x_data, y_data, epochs=100, batch_size=1, verbose=0)
m_eval = model.evaluate(x_data, y_data, verbose=0)
print(f'평가 결과: 손실={m_eval[0]:.4f}, 정확도={m_eval[1]:.4f}')

new_data = np.array([[1, 2.5], [10.5, 7.1]], dtype=np.float32)
pred = model.predict(new_data, verbose=0)
print('예측 확률: ', pred.ravel())
print('예측결과: ', [1 if i >= 0.5 else 0 for i in pred])
print('예측결과: ', (pred >= 0.5).astype(int).ravel())
print('예측결과: ', np.where(pred >= 0.5, 1, 0).ravel())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

평가 결과: 손실=0.6316, 정확도=0.6667
예측 확률:  [0.40696037 0.8513994 ]
예측결과:  [0, 1]
예측결과:  [0 1]
예측결과:  [0 1]


In [ ]:
# 모델 정의 방법2: Functional api
from numpy import shape
from tensorflow.keras.models import Model

input_layer = Input(shape=(2,))
output_layer = Dense(1, activation='sigmoid')(input_layer)
model2 = Model(inputs = input_layer, outputs = output_layer)

model2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

model2.summary()

model2.fit(x_data, y_data, epochs=100, batch_size=1, verbose=0)
m_eval = model2.evaluate(x_data, y_data, verbose=0)
print(f'평가 결과: 손실={m_eval[0]:.4f}, 정확도={m_eval[1]:.4f}')

new_data = np.array([[1, 2.5], [10.5, 7.1]], dtype=np.float32)
pred2 = model2.predict(new_data, verbose=0)

print('예측결과: ', np.where(pred2 >= 0.5, 1, 0).ravel())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

평가 결과: 손실=0.5881, 정확도=1.0000
예측결과:  [0 1]


In [ ]:
# 모델 정의 방법3: Subclassing api
class MyBinaryClass(Model):
    def __init__(self):
        super().__init__(name='MyBinaryClass')
        self.dense = Dense(1, activation='sigmoid', name='dense_sigmoid')
    
    def build(self, input_shape):
        # 첫번째 feed forward에서 가중치를 만든다. 
        super().build(input_shape)
    

    # fit(), evaluate(), predict() 하면 자동 호출
    def call(self, inputs, trainging=False):
        print('>>> call 실행됨, training= ', trainging)
        return self.dense(inputs)

model3 = MyBinaryClass()
model3.build(input_shape=(None, 2))
model3.summary()

model3.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])


model3.fit(x_data, y_data, epochs=100, batch_size=1, verbose=0)
m_eval = model3.evaluate(x_data, y_data, verbose=0)
print(f'평가 결과: 손실={m_eval[0]:.4f}, 정확도={m_eval[1]:.4f}')

new_data = np.array([[1, 2.5], [10.5, 7.1]], dtype=np.float32)
pred3 = model3.predict(new_data, verbose=0)

print('예측결과: ', np.where(pred3 >= 0.5, 1, 0).ravel())

Model: "MyBinaryClass"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_sigmoid (Dense)           │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

>>> call 실행됨, training=  False
>>> call 실행됨, training=  False
>>> call 실행됨, training=  False
평가 결과: 손실=0.3245, 정확도=1.0000
>>> call 실행됨, training=  False
예측결과:  [0 1]
